# Low-pass Filtering



In [ ]:
import h5py
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

from tqdne.conf import Config

In [ ]:
config = Config()
dataset_path = config.datasetdir / config.data_upsample_test
file = h5py.File(dataset_path, mode="r")
# load first 100 waveforms
waveforms = file["waveform"][0:100]
file.close()

## Filter Methods

In [ ]:
def filter(waveform):
    sos = signal.butter(**config.params_filter, fs=config.fs, output="sos")
    filtered = signal.sosfilt(sos, waveform)
    return filtered


def filter_with_noise(waveform):
    sigma = 0.01
    sos = signal.butter(**config.params_filter, fs=config.fs, output="sos")
    filtered = signal.sosfilt(sos, waveform) + np.random.randn(*waveform.shape) * sigma
    return filtered


filter_methods = {
    "Filter": filter,
    "Filter with noise": filter_with_noise,
}

## Sample Plot

In [ ]:
# Create a dropdown for selecting the filter method
filter_dropdown = widgets.Dropdown(
    options=filter_methods.keys(),
    value=list(filter_methods.keys())[0],
    description="Filter Method:",
)

# Create a dropdown for selecting the channel
channel_dropdown = widgets.Dropdown(
    options=list(range(len(waveforms[0]))),
    value=0,
    description="Channel:",
)

# Create a slider for selecting the waveform
waveform_slider = widgets.IntSlider(
    min=0,
    max=len(waveforms) - 1,
    step=1,
    value=0,
    description="Waveform:",
)


# Function to update the plot
def update_plot(filter_method, channel, waveform_index):
    waveform = waveforms[waveform_index]
    filter_fn = filter_methods[filter_method]
    filtered = filter_fn(waveform)

    fig, ax = plt.subplots(2, 1, figsize=(15, 8))
    ax[0].plot(waveform[channel])
    ax[0].set_title("Raw")
    ax[1].plot(filtered[channel])
    ax[1].set_title("Filtered")
    fig.subplots_adjust(hspace=0.5)
    fig.show()


# Create interactive plot
widgets.interactive(
    update_plot,
    filter_method=filter_dropdown,
    channel=channel_dropdown,
    waveform_index=waveform_slider,
)

## Power Spectral Density

In [ ]:
# Create a dropdown for selecting the filter method
filter_dropdown = widgets.Dropdown(
    options=filter_methods.keys(),
    value=list(filter_methods.keys())[0],
    description="Filter Method:",
)

# Create a dropdown for selecting the channel
channel_dropdown = widgets.Dropdown(
    options=list(range(len(waveforms[0]))),
    value=0,
    description="Channel:",
)

# Create boolean for selecting log scale
log_scale_checkbox = widgets.Checkbox(
    value=True,
    description="Log Scale",
)


# Function to update the plot
def update_plot(filter_method, channel, log_scale):
    filter_fn = filter_methods[filter_method]

    # compute power spectral density
    raw_fft = np.fft.rfft(waveforms[:, channel])
    raw_psd = np.abs(raw_fft) ** 2
    filtered_fft = np.fft.rfft(filter_fn(waveforms[:, channel]))
    filtered_psd = np.abs(filtered_fft) ** 2
    freqs = np.fft.rfftfreq(waveforms.shape[-1], 1 / config.fs)

    if log_scale:
        raw_psd = np.log(raw_psd)
        filtered_psd = np.log(filtered_psd)

    # compute mean and std
    raw_mean, raw_std = raw_psd.mean(axis=0), raw_psd.std(axis=0)
    filtered_mean, filtered_std = filtered_psd.mean(axis=0), filtered_psd.std(axis=0)

    # plot
    fig, ax = plt.subplots(1, 1, figsize=(15, 8))
    ax.plot(freqs, raw_mean, "r", label="Raw")
    ax.fill_between(freqs, raw_mean - raw_std, raw_mean + raw_std, color="r", alpha=0.2)
    ax.plot(freqs, filtered_mean, "b", label="Filtered")
    ax.fill_between(
        freqs,
        filtered_mean - filtered_std,
        filtered_mean + filtered_std,
        color="b",
        alpha=0.2,
    )
    fig.subplots_adjust(hspace=0.5)
    ax.set_xlabel("Frequency [Hz]")
    ax.set_ylabel("PSD [V**2/Hz]")
    ax.set_title("Power Spectral Density")
    ax.legend()
    fig.show()


# Create interactive plot
widgets.interactive(
    update_plot,
    filter_method=filter_dropdown,
    channel=channel_dropdown,
    log_scale=log_scale_checkbox,
)